# Introduction

This is the demo how to use AI Bridge [Ben](https://github.com/lorserker/ben) and [redeal](https://github.com/anntzer/redeal) compare Leading for 3NTs

* http://www.rpbridge.net/4g00.htm
* book [Winning Notrump Leads](https://www.amazon.com/Winning-Notrump-Leads-David-Bird/dp/1554947596) - redeal uses the idea from this book
* AI Ben models: https://huggingface.co/AIBridgeEngine

# Preparation

both needs [dds](https://github.com/dds-bridge/dds) library, here gives the hacked solution without recompiling.

It shall work smoothly in [colab](https://colab.research.google.com/), verified in 2023.11.18, which is ubuntu jammy with sudo permission


In [1]:
# install ben library & redeal library, both are hacked solution
!pip install --upgrade git+https://github.com/larrycai/ben-lite # install ben python package, not released
!pip install git+https://github.com/larrycai/redeal # install redeal without dds depend


  Cloning https://github.com/larrycai/ben-lite to /tmp/pip-req-build-ed_2bgu0
  Running command git clone --filter=blob:none --quiet https://github.com/larrycai/ben-lite /tmp/pip-req-build-ed_2bgu0
  Resolved https://github.com/larrycai/ben-lite to commit c167a2f1682d126519fa7cd933239b6408e972be
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/larrycai/redeal to /tmp/pip-req-build-44wezi5_
  Running command git clone --filter=blob:none --quiet https://github.com/larrycai/redeal /tmp/pip-req-build-44wezi5_
  Resolved https://github.com/larrycai/redeal to commit 94798f6cb61920e03fd6adb8579c7215e8736b3c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
# check unix environment
!cat /etc/os-release
!id
!pwd
!pip show ben

PRETTY_NAME="Ubuntu 22.04.3 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.3 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy
uid=0(root) gid=0(root) groups=0(root)
/content
Name: ben
Version: 0.2.1
Summary: ben
Home-page: 
Author: 
Author-email: 
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: bottle, gevent, grpcio-tools, jupyter, matplotlib, numpy, pythonnet, scikit-learn, scipy, tensorflow, tqdm, websockets
Required-by: 


In [18]:
# it is ubuntu & root @ https://colab.research.google.com/
!apt-get install -y libdds-dev
!dpkg -L libdds-dev
!cp /usr/lib/x86_64-linux-gnu/libdds.so /usr/local/lib/python3.10/dist-packages/redeal
# check location in pip show ben
#!git clone https://github.com/larrycai/dds-so.git  # clone and get libdds.so
#!cp dds-so/jammy/libdds.so /usr/local/lib/python3.10/dist-packages/redeal
#!ldd dds-so/jammy/libdds.so
#!find /usr/local/lib/python3.10/dist-packages | grep dds
#!export LD_LIBRARY_PATH=$HOME/dds-so/jammy:$LD_LIBRARY_PATH
# !export LD_LIBRARY_PATH=/usr/lib/x86_64-linux-gnu:$LD_LIBRARY_PATH
import os
print(os.name)
from redeal import dds
#!cp dds-so/jammy/libdds.so /opt/conda/lib/python3.11/site-packages/redeal

	linux-vdso.so.1 (0x00007fff4bdf0000)
	libboost_thread.so.1.74.0 => /lib/x86_64-linux-gnu/libboost_thread.so.1.74.0 (0x00007df1378e7000)
	libstdc++.so.6 => /lib/x86_64-linux-gnu/libstdc++.so.6 (0x00007df1376bb000)
	libm.so.6 => /lib/x86_64-linux-gnu/libm.so.6 (0x00007df1375d4000)
	libgomp.so.1 => /lib/x86_64-linux-gnu/libgomp.so.1 (0x00007df13758a000)
	libgcc_s.so.1 => /lib/x86_64-linux-gnu/libgcc_s.so.1 (0x00007df13756a000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007df137340000)
	/lib64/ld-linux-x86-64.so.2 (0x00007df137b7b000)
/usr/local/lib/python3.10/dist-packages/ben/libdds.so
/usr/local/lib/python3.10/dist-packages/ben/ddsolver
/usr/local/lib/python3.10/dist-packages/ben/ddsolver/__pycache__
/usr/local/lib/python3.10/dist-packages/ben/ddsolver/__pycache__/ddsolver.cpython-310.pyc
/usr/local/lib/python3.10/dist-packages/ben/ddsolver/__pycache__/dds.cpython-310.pyc
/usr/local/lib/python3.10/dist-packages/ben/ddsolver/ddsolver.py
/usr/local/lib/python3.10/dist-packages/be

# Let's see the question

https://bridgeonline.ru/wp-content/uploads/2020/04/Winning_Notrump_Leads.pdf (googled, ok to use?)

Chapter 1, Hand 3

````
Bidding: (None all)
 West  North   East  South
                     1NT
 pass  3NT     all pass

Sit in west: Which is the best lead

 South: 15-17point, 5332, 4432, 4333
````

What would you lead from: ♠ J 8 6 ♥ 9 5 ♦ K Q 7 2 ♣ K J 7 2 ?

# Redeal's solution

Thomas Andrew's Deal is a deal generator: it outputs deals satisfying whatever conditions you specify, then estimate which is best by using dds

In [21]:
from redeal import *
from redeal import dds
from redeal.global_defs import Seat, Suit, Card, Rank, Strain
def accept(deal):
    north, south = deal.north, deal.south
    return (
        balanced(south) and       # south
        15 <= south.hcp <= 17 and # south
        len(north.spades) <4 and len(north.hearts) < 4 and # north
        9 <= north.hcp <= 11                               # north
    )

N_TRIES=5000
def nt_leader(predeal, del_lst=[], n_tries=N_TRIES):
    samples = []
    dealer = Deal.prepare(predeal)

    balanced = Shape("(4333)") + Shape("(4432)") + Shape("(5332)")



    simulation = OpeningLeadSim(accept, "3NS", imps)
    simulation.initial(dealer)

    found = 0
    for _ in range(1000*n_tries):
        if found >= n_tries:
            break
        deal = dealer()
        if not accept(deal):
            continue

        found += 1
        score_3NS = deal.dd_score('3NS')
        if score_3NS < 0:
            # it is down
            samples.append(deal)
        simulation.do(deal)

    # remove uselss
    # check https://github.com/anntzer/redeal/blob/master/redeal/redeal.py#L623
    for card in del_lst:
        simulation.payoff.entries.remove(Card.from_str(card))

    print(f"Result of simulation: {found} deals")
    simulation.final(n_tries)
    return samples

predeal = {"W": "J86 95 KQ72 KJ72"}
print(predeal['W'])

cards_needs_removed=["SJ", "H5"]
samples = nt_leader(predeal,cards_needs_removed)

deal = samples[0]
Deal.set_str_style('long')
print("Check one sample")
print("3NT sample:", deal)

result=deal.dd_all_tricks("N" , "W")
score_3NS = deal.dd_score('3NS')


J86 95 KQ72 KJ72


Exception: ignored

# AI Ben's solution

It uses AI Ben machine learning models to find the best leader


In [22]:
import huggingface_hub
import os
lead_id="AIBridgeEngine/Ben-3B-Lead-v0.1"
bidding_id="AIBridgeEngine/Ben-3B-Bidding-v0.1"
biddinginfo_id="AIBridgeEngine/Ben-3B-Biddinginformation-v0.1"
singledummy_id="AIBridgeEngine/Ben-3B-Singledummy-v0.1"
folder="hfmodels"
huggingface_hub.snapshot_download(repo_id=lead_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=bidding_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=biddinginfo_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=singledummy_id, local_dir=folder)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

lead-1000000.meta:   0%|          | 0.00/78.6k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

lead-1000000.index:   0%|          | 0.00/546 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

...

lr3-1000000.index:   0%|          | 0.00/659 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/211 [00:00<?, ?B/s]

lr3-1000000.meta:   0%|          | 0.00/92.4k [00:00<?, ?B/s]

lr3-1000000.data-00000-of-00001:   0%|          | 0.00/10.5M [00:00<?, ?B/s]

'/content/hfmodels'

In [23]:
from ben.nn.models import Models
from ben.bots import BotLead
from ben.sample import Sample
from ben.nn.leader import Leader
from ben.nn.bid_info import BidInfo
from ben.nn.bidder import Bidder
from ben.nn.lead_singledummy import LeadSingleDummy
lead = Leader(f'{folder}/lead-1000000')
biddinginfo = BidInfo(f'{folder}/binfo-100000')
bidder = Bidder("", f'{folder}/bidding-100000')
sd_model = LeadSingleDummy(f'{folder}/lr3-1000000')
models = Models(bidder, biddinginfo, lead, sd_model, None, 0.1, 0.05,0.75, 0.66)
sampler = Sample(0.01, 0.1, 0.03, 0.05, 64, 5000, 5000, 128, 100, False)


Instructions for updating:
non-resource variables are not supported in the long term


In [25]:
# both vulnerable. you are sitting North as dealer and you hold
hand = 'J86.950.KQ72.KJ72'

# the auction goes:
auction = ['PAD_START','PAD_START','PAD_START',"1N","PASS",'PASS', 'PASS', 'PASS']

# what to lead?

lead_bot = BotLead([True, True], hand, models, -1, -1, models.lead_threshold, sampler, False)
lead = lead_bot.find_opening_lead(auction)

Sorting by insta_score
Cx
0.4442762565795649


In [26]:
lead.card

S6

seems like the engine chose to lead the ace of diamonds

the other options it considered were: a small spade and a small club

In [27]:
lead.to_dict()['candidates']

[{'card': 'Sx',
  'insta_score': 0.168,
  'expected_tricks': 7.72,
  'p_make_contract': 0.76},
 {'card': 'Cx',
  'insta_score': 0.4443,
  'expected_tricks': 7.92,
  'p_make_contract': 0.78},
 {'card': 'Dx',
  'insta_score': 0.3827,
  'expected_tricks': 8.17,
  'p_make_contract': 0.82}]

in the above output:
- `insta_score` reflects the preference of the neural network
- `expected_tricks` how many tricks declarer is expected to take on this lead
- `p_make_contract` is the probability of the contract making on this lead

the probability of making and the expected tricks are computed on samples which are consistent with the auction. the samples are estimated single dummy using a neural network (more on this in another tutorial). we could also solve the samples double dummy, but that would be quite a bit slower.

In [28]:
# each row is one sample board
# the hands are in the order: LHO, Partner, RHO. Your cards are not shown as they are fixed/

lead.samples

['T9xx.KQ8.Txx.8xx AKxx.JTx.xx.Txx Qx.Axxx.AJ98.AQ9',
 'Ax.Tx.J98xx.T8x KT9xxx.KJxx.x.xx Qx.AQ8x.ATx.AQ9x',
 'Txxx.KJx.ATx.8x Kx.AT8xx.8x.T9xx AQ9x.Qx.J9xx.AQx']